<h2><center>THE CHINESE UNIVERSITY OF HONG KONG<br /> 
    Department of Mathematics<br />
 MATH4280<br />
    Innovation and Design in Big Data Analytics<br />
    Sample test </center></h2>
    


## Question 1

You are now given a dataset, containing a three dimensional time series. The coordinates are given as `x`,`y` and `z`. The series has `10000` time steps. 

In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np
df=pd.read_csv('Q1.csv')
data=np.array(df.iloc[0:100000:10,1:]).T
fig=px.line_3d(x=data[0],y=data[1],z=data[2])
fig.show()

We now would like to do regression of the trajectory in the following form:

Suppose ${\bf D}= [d_0,d_1,\cdots,d_{9999}]$ is the time series, where $d_t = [x_t,y_t,z_t]^T$ denotes the coordinates at time $t$. We want to use a neural network $\Phi_{NN}:\mathbb{R}^3 \rightarrow \mathbb{R}^3$ such that 
$$\Phi_{NN}([x_t,y_t,z_t]^T)=[x_{t+1},y_{t+1},z_{t+1}]^T,$$
to predict the coordinate of the next step.

We have the following requirement:
- Training-Validating-Testing:

1. Use the first 8000 steps as the training data. (i.e. the training input-output pairs are $[(d_0,d_1),(d_1,d_2),\cdots,(d_{7999},d_{8000})]$). 
2. Use `0.2` for `validation_split`. 

3. Use the remaining steps as the testing data.

4. Training Epoch should be not greater than `1000` and batchsize=`200`.

- Performance:

1. Attain an MSE (on the testing data) less than `0.01`.

The MSE should be computed by this:

Denote your prediction by $P=[p_{8000},p_{8001},\cdots,p_{9999}]$.
The MSE is defined by 
$$\frac{1}{2000}\sum_{k=8000}^{9999}(p_k-d_k)^2$$



2. Save your model as a `h5` file (named `q1a.h5`) and submit it with the jupyter notebook.


In [ ]:
## answer of 1a


Plot the training loss and validation loss (MSE).

In [ ]:
## answer of 1b


But in reality, if we want to do long-term predictions (next few steps instead just one step), we need to use our previous predictions for the future ones. 

Now, train a neural network under the same requirement above to predict the trajectory for the next 100 steps ($t=8001,\cdots,8100$) using the same training set, so that the MSE is less than `100`.

In other words, for $t\geq 8001$ (testing phase), your prediction should base on your previous prediction, instead of the testing data.

i.e. $$p_{t+1}=\Phi_{NN}(p_{t})$$
but NOT
$$p_{t+1}=\Phi_{NN}(d_{t})$$

Predict the trajectory for $t=8001,\cdots,8100$ and calculate the mean squared error.

Save your model as `q1b.h5` and submit it with the jupyter notebook.

In [ ]:
## answer of 1c


In [ ]:
from tensorflow.keras.models import load_model
import plotly.graph_objects as go
import numpy as np

model = load_model('q1b.h5')
in_tmp = data[:, 8000].T
pred = np.zeros((3, 100))

for t, _ in enumerate(range(8000, 8100)):
    in_tmp = model.predict(in_tmp.reshape(1, -1), verbose=0)
    pred[:, t] = in_tmp

plot_fig = go.Figure(data=go.Scatter3d(
    x=pred[0], y=pred[1], z=pred[2], mode='lines',
    line=dict(color='blue')
))

actual_traj = data[:, 8001:8101]
plot_fig.add_trace(go.Scatter3d(
    x=actual_traj[0], y=actual_traj[1], z=actual_traj[2], mode='lines',
    line=dict(color='red')
))

print(f'MSE: {((pred[:,:100] - actual_traj) ** 2).mean()}')
plot_fig.show()
